<a href="https://colab.research.google.com/github/djmayo04/AI_Mini-Project2/blob/main/notebooks/AI_MP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-groq groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.1 MB/s eta 0:00:00


In [19]:
import os

REPO_URL = "https://github.com/djmayo04/AI_Mini-Project2.git"
REPO_DIR = "AI_Mini-Project2"

%cd /content

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}
else:
    %cd {REPO_DIR}
    !git pull
    %cd /content

%cd {REPO_DIR}
!ls

/content
Cloning into 'AI_Mini-Project2'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 64 (delta 25), reused 5 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 23.12 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/AI_Mini-Project2
agent.py       memory.py  README.md	       tools.py
llm_helper.py  notebooks  search_documents.py


In [3]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
print(os.environ.get("GROQ_API_KEY") is not None)

True


In [20]:
from llm_helper import get_completion
from agent import run_agent

print(get_completion("Say hello in one sentence."))
print(run_agent("What does the syllabus say about deadlines?"))

Hello!
According to the syllabus, **all course deadlines are fixed** and must be met unless you obtain prior written permission from the instructor.  

### Key points about deadlines

| Item | Deadline policy |
|------|-----------------|
| **Assignments & Projects** | Each assignment has a specific due date listed in the schedule. Late submissions are accepted only with an approved extension; otherwise a penalty of **10 % per day** (up to a maximum of 50 %) will be applied. |
| **Quizzes & Exams** | These must be completed at the scheduled time. No make‑up quizzes or exams are offered except for documented emergencies (e.g., medical, family crisis). |
| **Discussion posts** | Must be posted by the **midnight deadline** on the due day. Posts submitted after the deadline receive a **5 %** reduction in the participation grade. |
| **Final Paper/Project** | The final submission is due **Friday, May 15, 2026, 11:59 PM**. No extensions will be granted after this date. |
| **Extensions** | To